In [1]:
# Block 1 — Login & Lấy dữ liệu tất cả HOSE/HNX/UPCOM
import pandas as pd
from FiinQuantX import FiinSession, BarDataUpdate

# --- Login ---
username = "DSTC_18@fiinquant.vn"
password = "Fiinquant0606"

client = FiinSession(
    username=username,
    password=password
).login()

# --- Lấy danh sách cổ phiếu từng sàn ---
tickers_hose  = list(client.TickerList(ticker="VNINDEX"))     # HOSE
print(f"Số mã HOSE: {len(tickers_hose)}")

# --- Lấy dữ liệu lịch sử toàn bộ (có thể nặng, nên lấy theo batch nếu cần) ---
event_history = client.Fetch_Trading_Data(
    realtime=False,
    tickers=tickers_hose,
    fields=['open','high','low','close','volume','bu','sd','fs','fn'], 
    adjusted=True,
    by="1d",
    from_date="2023-01-01"   # backtest từ 2023 tới nay
)

df_all = event_history.get_data()
print("History ban đầu:", df_all.head())

# --- Callback realtime ---
def onDataUpdate(data: BarDataUpdate):
    global df_all
    df_update = data.to_dataFrame()
    df_all = pd.concat([df_all, df_update])
    df_all = df_all.drop_duplicates()
    print("Realtime update:")
    print(df_update.head())

# --- Bật realtime nối tiếp dữ liệu ---
event_realtime = client.Fetch_Trading_Data(
    realtime=True,
    tickers=tickers_hose,
    fields=['open','high','low','close','volume','bu','sd','fs','fn'], 
    adjusted=True,
    by="1d",
    period=1,
    callback=onDataUpdate
)


Số mã HOSE: 415
Fetching data, it may take a while. Please wait...
History ban đầu:   ticker         timestamp      open      high       low     close     volume  \
0    AAA  2023-01-03 00:00  6539.643  6866.145  6539.643  6866.145  1543984.0   
1    AAA  2023-01-04 00:00  6866.145  7000.587  6827.733  6827.733  1302505.0   
2    AAA  2023-01-05 00:00  6866.145  6904.557  6808.527  6885.351   980473.0   
3    AAA  2023-01-06 00:00  6885.351  6990.984  6818.130  6856.542  1431699.0   
4    AAA  2023-01-09 00:00  6914.160  6962.175  6760.512  6789.321  1121385.0   

         bu        sd           fs           fn  
0  938600.0  504700.0   40579000.0  899404000.0  
1  462900.0  780600.0  151639000.0   36850000.0  
2  487200.0  473700.0  343911000.0  -59103000.0  
3  564300.0  828300.0  345999000.0 -294312000.0  
4  414000.0  631800.0  514557000.0 -483197000.0  


In [2]:
# Block 2 — Lấy dữ liệu FA theo quý (HOSE only)

def fetch_fa_quarterly(ticker, latest_year=2025, n_periods=32):
    try:
        fi_list = client.FundamentalAnalysis().get_ratios(
            tickers=[ticker],
            TimeFilter="Quarterly",
            LatestYear=latest_year,
            NumberOfPeriod=n_periods,
            Consolidated=True
        )

        # Nếu không có dữ liệu thì bỏ qua
        if not fi_list or not isinstance(fi_list, list):
            return pd.DataFrame()

        df = pd.DataFrame(fi_list)
        if df.empty:
            return pd.DataFrame()

        df["ticker"] = ticker
        if "ReportDate" in df.columns:
            df["ReportDate"] = pd.to_datetime(df["ReportDate"])
        else:
            # Nếu không có ReportDate thì tạo cột null để tránh lỗi concat
            df["ReportDate"] = pd.NaT

        return df

    except Exception as e:
        print(f"⚠️ Lỗi khi lấy FA cho {ticker}: {e}")
        return pd.DataFrame()


# --- Lọc danh sách: chỉ giữ những mã có dữ liệu FA ---
fa_list = []
valid_tickers = []

for t in tickers_hose:   # lấy theo danh sách HOSE từ Block 1
    df_fa = fetch_fa_quarterly(t, latest_year=2025, n_periods=32)
    if not df_fa.empty:
        fa_list.append(df_fa)
        valid_tickers.append(t)

# --- Gộp DataFrame ---
if fa_list:
    fa_data = pd.concat(fa_list, ignore_index=True)
else:
    fa_data = pd.DataFrame()

print(f"Số mã HOSE ban đầu: {len(tickers_hose)}")
print(f"Số mã có dữ liệu FA: {len(valid_tickers)}")
print("FA Data sample:")
print(fa_data.head())


⚠️ Lỗi khi lấy FA cho FUETPVND: 'FUETPVND'
Số mã HOSE ban đầu: 415
Số mã có dữ liệu FA: 392
FA Data sample:
   organizationId ticker  year  quarter  \
0          894364    CCC  2023        4   
1          894364    CCC  2024        1   
2          894364    CCC  2024        2   
3          894364    CCC  2024        3   
4          894364    CCC  2024        4   

                                              ratios ReportDate  
0  {'SolvencyRatio': {'DebtToEquityRatio': 1.5102...        NaT  
1  {'SolvencyRatio': {'DebtToEquityRatio': 0.7722...        NaT  
2  {'SolvencyRatio': {'DebtToEquityRatio': 0.7357...        NaT  
3  {'SolvencyRatio': {'DebtToEquityRatio': 0.7914...        NaT  
4  {'SolvencyRatio': {'DebtToEquityRatio': 0.6437...        NaT  


In [3]:
# Block 3 — Chuẩn hoá FA + Merge với giá (HOSE only, dựa theo Block 2)

import pandas as pd

# --- Các chỉ số FA cần lấy ---
fa_fields = [
    "DebtToEquityRatio","EBITMargin","ROA","ROE","ROIC",
    "BasicEPS","PriceToBook","PriceToEarning",
    "NetRevenueGrowthYoY","GrossProfitGrowthYoY"
]

# --- Hàm nổ ratios ---
def explode_ratios(df, fa_fields):
    records = []
    for _, row in df.iterrows():
        d = {
            "ticker": row["ticker"],
            "fa_year": int(row["year"]),
            "fa_quarter": int(row["quarter"])
        }
        ratios = row.get("ratios", {})
        if isinstance(ratios, dict):   # ✅ fix chỗ lỗi
            for f in fa_fields:
                val = None
                for section in ratios.values():
                    if isinstance(section, dict) and f in section:
                        val = section[f]
                d[f] = val
        else:
            # nếu ratios không phải dict thì gán NaN hết
            for f in fa_fields:
                d[f] = None
        records.append(d)
    return pd.DataFrame(records)

# --- Chuẩn hoá FA ---
fa_clean = explode_ratios(fa_data, fa_fields)

# --- Chuẩn hoá giá ---
df_price = df_all[df_all["ticker"].isin(valid_tickers)].copy()
df_price["timestamp"] = pd.to_datetime(df_price["timestamp"])
df_price = df_price.sort_values(["ticker","timestamp"])

# tạo key (fa_year, fa_quarter) = quý trước
pi = df_price["timestamp"].dt.to_period("Q")
prev_pi = pi - 1
df_price["fa_year"] = prev_pi.dt.year.astype(int)
df_price["fa_quarter"] = prev_pi.dt.quarter.astype(int)

# --- Xử lý FA: giữ duy nhất bản cuối cùng mỗi quý
fa_clean = (
    fa_clean.sort_values(["ticker","fa_year","fa_quarter"])
            .drop_duplicates(subset=["ticker","fa_year","fa_quarter"], keep="last")
)

# --- Merge giá + FA ---
df_merged = df_price.merge(
    fa_clean,
    on=["ticker","fa_year","fa_quarter"],
    how="left"
)

# FFill theo thời gian trong từng ticker để lấp chỗ trống
df_merged = df_merged.sort_values(["ticker","timestamp"])
df_merged[fa_fields] = df_merged.groupby("ticker")[fa_fields].ffill()

print("Sample merged:")
print(df_merged.head())
print("Số mã merge thành công:", df_merged["ticker"].nunique())




Sample merged:
  ticker  timestamp      open      high       low     close     volume  \
0    AAA 2023-01-03  6539.643  6866.145  6539.643  6866.145  1543984.0   
1    AAA 2023-01-04  6866.145  7000.587  6827.733  6827.733  1302505.0   
2    AAA 2023-01-05  6866.145  6904.557  6808.527  6885.351   980473.0   
3    AAA 2023-01-06  6885.351  6990.984  6818.130  6856.542  1431699.0   
4    AAA 2023-01-09  6914.160  6962.175  6760.512  6789.321  1121385.0   

         bu        sd           fs  ...  DebtToEquityRatio  EBITMargin  \
0  938600.0  504700.0   40579000.0  ...           0.507521   -0.049731   
1  462900.0  780600.0  151639000.0  ...           0.507521   -0.049731   
2  487200.0  473700.0  343911000.0  ...           0.507521   -0.049731   
3  564300.0  828300.0  345999000.0  ...           0.507521   -0.049731   
4  414000.0  631800.0  514557000.0  ...           0.507521   -0.049731   

        ROA       ROE      ROIC    BasicEPS  PriceToBook  PriceToEarning  \
0  0.014669  0.0295

In [4]:
import gc
del df_all
gc.collect()


0

In [5]:
# Block 4 — Tính các chỉ số TA (trên df_merged từ Block 3)

import pandas as pd

# --- Khởi tạo Indicator ---
fi = client.FiinIndicator()

# --- Hàm tính TA theo từng ticker ---
def add_ta_indicators(df):
    df = df.sort_values("timestamp").copy()

    # EMA
    df['ema_5']  = fi.ema(df['close'], window=5)
    df['ema_20'] = fi.ema(df['close'], window=20)
    df['ema_50'] = fi.ema(df['close'], window=50)

    # MACD
    df['macd']        = fi.macd(df['close'], window_fast=12, window_slow=26)
    df['macd_signal'] = fi.macd_signal(df['close'], window_fast=12, window_slow=26, window_sign=9)
    df['macd_diff']   = fi.macd_diff(df['close'], window_fast=12, window_slow=26, window_sign=9)

    # RSI
    df['rsi'] = fi.rsi(df['close'], window=14)

    # Bollinger Bands
    df['bollinger_hband'] = fi.bollinger_hband(df['close'], window=20, window_dev=2)
    df['bollinger_lband'] = fi.bollinger_lband(df['close'], window=20, window_dev=2)

    # ATR
    df['atr'] = fi.atr(df['high'], df['low'], df['close'], window=14)

    # OBV
    df['obv'] = fi.obv(df['close'], df['volume'])

    # VWAP
    df['vwap'] = fi.vwap(df['high'], df['low'], df['close'], df['volume'], window=14)

    return df

# --- Áp dụng cho toàn bộ df_merged ---
df_with_ta = df_merged.groupby("ticker", group_keys=False).apply(add_ta_indicators)

print("Sample with TA:")
print(df_with_ta.head())
print("Shape sau khi thêm TA:", df_with_ta.shape)


Sample with TA:
  ticker  timestamp      open      high       low     close     volume  \
0    AAA 2023-01-03  6539.643  6866.145  6539.643  6866.145  1543984.0   
1    AAA 2023-01-04  6866.145  7000.587  6827.733  6827.733  1302505.0   
2    AAA 2023-01-05  6866.145  6904.557  6808.527  6885.351   980473.0   
3    AAA 2023-01-06  6885.351  6990.984  6818.130  6856.542  1431699.0   
4    AAA 2023-01-09  6914.160  6962.175  6760.512  6789.321  1121385.0   

         bu        sd           fs  ...  ema_50  macd  macd_signal  macd_diff  \
0  938600.0  504700.0   40579000.0  ...     NaN   NaN          NaN        NaN   
1  462900.0  780600.0  151639000.0  ...     NaN   NaN          NaN        NaN   
2  487200.0  473700.0  343911000.0  ...     NaN   NaN          NaN        NaN   
3  564300.0  828300.0  345999000.0  ...     NaN   NaN          NaN        NaN   
4  414000.0  631800.0  514557000.0  ...     NaN   NaN          NaN        NaN   

   rsi  bollinger_hband  bollinger_lband  atr       

In [6]:
import gc
del df_merged
gc.collect()

0

In [7]:
# Block 5 — Feature engineering & scaling

import numpy as np

# --- Danh sách cột FA & TA ---
fa_features = [
    "DebtToEquityRatio","EBITMargin","ROA","ROE","ROIC",
    "BasicEPS","PriceToBook","PriceToEarning",
    "NetRevenueGrowthYoY","GrossProfitGrowthYoY"
]

ta_features = [
    "ema_5","ema_20","ema_50","macd","macd_signal","macd_diff",
    "rsi","bollinger_hband","bollinger_lband","atr","obv","vwap"
]

# --- Chuẩn hoá FA: cross-section min-max scaling theo ngày ---
def scale_fa_minmax(df):
    df_scaled = df.copy()
    for f in fa_features:
        vals = df[f].astype(float)
        vmin, vmax = vals.min(), vals.max()
        if np.isfinite(vmin) and np.isfinite(vmax) and vmax > vmin:
            df_scaled[f] = (vals - vmin) / (vmax - vmin)
        else:
            df_scaled[f] = np.nan
    return df_scaled

df_scaled_fa = df_with_ta.groupby("timestamp", group_keys=False).apply(scale_fa_minmax)

# --- Chuẩn hoá TA: rolling z-score theo từng ticker ---
def zscore_rolling(series, window=60):
    return (series - series.rolling(window).mean()) / series.rolling(window).std()

df_scaled = df_scaled_fa.groupby("ticker", group_keys=False).apply(
    lambda g: g.assign(**{f"{col}_z": zscore_rolling(g[col], 60) for col in ta_features})
)

# --- Drop các cột gốc TA, giữ bản z-score ---
keep_cols = ["ticker","timestamp"] + fa_features + [f"{col}_z" for col in ta_features]
df_features = df_scaled[keep_cols].dropna().reset_index(drop=True)

print("Sample features:")
print(df_features.head())
print("Shape sau khi scaling & dropna:", df_features.shape)


Sample features:
  ticker  timestamp  DebtToEquityRatio  EBITMargin       ROA       ROE  \
0    AAA 2023-06-14           0.559115    0.978956  0.368731  0.415728   
1    AAA 2023-06-15           0.559115    0.978956  0.368731  0.415728   
2    AAA 2023-06-16           0.559115    0.978956  0.368731  0.415728   
3    AAA 2023-06-19           0.559115    0.978956  0.368731  0.415728   
4    AAA 2023-06-20           0.559115    0.978956  0.368731  0.415728   

     ROIC  BasicEPS  PriceToBook  PriceToEarning  ...  ema_50_z    macd_z  \
0  0.7533  0.158046      0.37764        0.537353  ...  1.799046 -0.008320   
1  0.7533  0.158046      0.37764        0.537353  ...  1.761009 -0.314808   
2  0.7533  0.158046      0.37764        0.537353  ...  1.701399 -0.891565   
3  0.7533  0.158046      0.37764        0.537353  ...  1.651219 -1.278916   
4  0.7533  0.158046      0.37764        0.537353  ...  1.609327 -1.509982   

   macd_signal_z  macd_diff_z     rsi_z  bollinger_hband_z  bollinger_lband

In [8]:
del df_with_ta, df_scaled, df_scaled_fa
gc.collect()


0

In [9]:
# Block 6 — Dimensionality reduction & Clustering (t-SNE + DBSCAN)

from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN

# --- Chọn các cột features để clustering ---
feature_cols = [
    "DebtToEquityRatio","EBITMargin","ROA","ROE","ROIC",
    "BasicEPS","PriceToBook","PriceToEarning",
    "NetRevenueGrowthYoY","GrossProfitGrowthYoY"
] + [c for c in df_features.columns if c.endswith("_z")]

# --- Thêm cột tháng để snapshot ---
df_features["month"] = df_features["timestamp"].dt.to_period("M")

cluster_results = []

for (month, g) in df_features.groupby("month"):
    if len(g) < 10:   # quá ít cổ phiếu thì bỏ
        continue

    X = g[feature_cols].values

    # --- t-SNE giảm chiều còn 2D ---
    tsne = TSNE(n_components=2, perplexity=30, learning_rate="auto", init="random", random_state=42)
    X_emb = tsne.fit_transform(X)

    # --- DBSCAN clustering ---
    db = DBSCAN(eps=0.5, min_samples=5).fit(X_emb)
    labels = db.labels_

    temp = g[["ticker","timestamp"]].copy()
    temp["cluster"] = labels
    temp["tsne_x"] = X_emb[:,0]
    temp["tsne_y"] = X_emb[:,1]
    temp["month"]  = str(month)

    cluster_results.append(temp)

df_clusters = pd.concat(cluster_results, ignore_index=True)

print("Cluster sample:")
print(df_clusters.head())
print("Số cụm mỗi tháng:")
print(df_clusters.groupby("month")["cluster"].nunique())


Cluster sample:
  ticker  timestamp  cluster     tsne_x     tsne_y    month
0    AAA 2023-06-14       -1  34.417667 -21.641058  2023-06
1    AAA 2023-06-15       -1  34.361568 -21.848602  2023-06
2    AAA 2023-06-16       -1  35.918636 -42.906532  2023-06
3    AAA 2023-06-19       -1  35.490566 -42.649422  2023-06
4    AAA 2023-06-20       -1  34.553066 -42.459969  2023-06
Số cụm mỗi tháng:
month
2023-06     16
2023-07     67
2023-08     78
2023-09     40
2023-10     83
2023-11     55
2023-12     83
2024-01     90
2024-02     31
2024-03     64
2024-04     43
2024-05     66
2024-06     94
2024-07     87
2024-08     65
2024-09     86
2024-10    104
2024-11     80
2024-12     74
2025-01     49
2025-02     68
2025-03     93
2025-04     52
2025-05     79
2025-06     95
2025-07    110
2025-08     74
Name: cluster, dtype: int64


In [10]:
#block 7
import numpy as np
import pandas as pd
import os, gc, json

LOOKBACK = 64   # window size
DATA_DIR = "./tensors/"
os.makedirs(DATA_DIR, exist_ok=True)

# --- chọn feature columns (bỏ các cột không phải feature) ---
feature_cols = [c for c in df_features.columns if c not in ["ticker","timestamp","cluster","month"]]

tensor_index = []

# --- Lặp qua từng cluster ---
for c_id, g in df_clusters.groupby("cluster"):
    if c_id == -1:   # DBSCAN noise bỏ qua
        continue

    tickers = sorted(g["ticker"].unique())
    g_feat = df_features[df_features["ticker"].isin(tickers)].copy()

    # Pivot: index = timestamp, columns = MultiIndex (ticker, feature)
    pivoted = g_feat.pivot(index="timestamp", columns="ticker", values=feature_cols)
    pivoted.columns = pd.MultiIndex.from_product([tickers, feature_cols])

    # Mask: 1 = có dữ liệu, 0 = NaN
    mask_df = ~pivoted.isna()

    # Fill NaN để reshape được (mask vẫn giữ thông tin missing)
    pivoted_filled = pivoted.ffill().bfill()

    T, N, F = len(pivoted_filled.index), len(tickers), len(feature_cols)

    X = pivoted_filled.values.reshape(T, N, F)
    M = mask_df.values.reshape(T, N, F).astype(int)

    cluster_tensors, cluster_masks = [], []
    for i in range(LOOKBACK, T):
        cluster_tensors.append(X[i-LOOKBACK:i])
        cluster_masks.append(M[i-LOOKBACK:i])

    if cluster_tensors:
        X_arr, M_arr = np.array(cluster_tensors), np.array(cluster_masks)

        # Save file
        tensor_file = f"cluster_{c_id}_tensor.npy"
        mask_file   = f"cluster_{c_id}_mask.npy"
        np.save(os.path.join(DATA_DIR, tensor_file), X_arr)
        np.save(os.path.join(DATA_DIR, mask_file), M_arr)

        tensor_index.append({
            "cluster": int(c_id),
            "tickers": tickers,
            "dates": [str(d) for d in pivoted_filled.index[LOOKBACK:]],
            "tensor_file": tensor_file,
            "mask_file": mask_file
        })

        print(f"Cluster {c_id}: tensor {X_arr.shape}, mask {M_arr.shape} saved.")

    # Giải phóng RAM
    del g_feat, pivoted, pivoted_filled, mask_df, X, M, cluster_tensors, cluster_masks
    gc.collect()

# --- Lưu metadata ---
with open(os.path.join(DATA_DIR, "tensor_index.json"), "w") as f:
    json.dump(tensor_index, f, indent=2)

print("✅ Done Block 7: tensors + masks saved for all clusters.")

# Sau Block 7 có thể xoá df_features cho nhẹ RAM
del df_features
gc.collect()


Cluster 0: tensor (490, 64, 21, 22), mask (490, 64, 21, 22) saved.
Cluster 1: tensor (490, 64, 15, 22), mask (490, 64, 15, 22) saved.
Cluster 2: tensor (490, 64, 15, 22), mask (490, 64, 15, 22) saved.
Cluster 3: tensor (490, 64, 17, 22), mask (490, 64, 17, 22) saved.
Cluster 4: tensor (490, 64, 19, 22), mask (490, 64, 19, 22) saved.
Cluster 5: tensor (490, 64, 21, 22), mask (490, 64, 21, 22) saved.
Cluster 6: tensor (490, 64, 23, 22), mask (490, 64, 23, 22) saved.
Cluster 7: tensor (490, 64, 25, 22), mask (490, 64, 25, 22) saved.
Cluster 8: tensor (490, 64, 21, 22), mask (490, 64, 21, 22) saved.
Cluster 9: tensor (490, 64, 22, 22), mask (490, 64, 22, 22) saved.
Cluster 10: tensor (490, 64, 23, 22), mask (490, 64, 23, 22) saved.
Cluster 11: tensor (490, 64, 19, 22), mask (490, 64, 19, 22) saved.
Cluster 12: tensor (490, 64, 28, 22), mask (490, 64, 28, 22) saved.
Cluster 13: tensor (490, 64, 29, 22), mask (490, 64, 29, 22) saved.
Cluster 14: tensor (490, 64, 25, 22), mask (490, 64, 25, 2

56

In [11]:
# Block 7.5 — Chuẩn bị dữ liệu backtest cho reward thật
import gc

# Chỉ giữ dữ liệu cần thiết để tính reward (close price)
# df_price có từ Block 1 (OHLCV đầy đủ)
df_backtest = df_price[["ticker", "timestamp", "close"]].copy()

# Ép timestamp về dạng datetime để đồng bộ
df_backtest["timestamp"] = pd.to_datetime(df_backtest["timestamp"])

print("✅ Done Block 7.5: df_backtest sẵn sàng cho reward.")
print("Kích thước df_backtest:", df_backtest.shape)
print("Tickers unique:", df_backtest["ticker"].nunique())

# Xóa những biến không còn cần để tiết kiệm RAM
del df_price
gc.collect()


✅ Done Block 7.5: df_backtest sẵn sàng cho reward.
Kích thước df_backtest: (256151, 3)
Tickers unique: 391


21

In [14]:
# Block 8 — A3C multi-stock per-cluster (save models + signals)
import os, gc, json, csv
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

DATA_DIR = "./tensors/"
SIG_DIR  = "./signals/"
MODEL_DIR = "./models/"
os.makedirs(SIG_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

SIG_FILE = os.path.join(SIG_DIR, "a3c_signals.csv")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Reset signals file ---
if os.path.exists(SIG_FILE):
    os.remove(SIG_FILE)
with open(SIG_FILE, "w", newline="") as f:
    csv.writer(f).writerow(["date","ticker","signal"])

# --- Load metadata ---
with open(os.path.join(DATA_DIR, "tensor_index.json"), "r") as f:
    tensor_index = json.load(f)

# --- Model ---
class A3CNet(nn.Module):
    def __init__(self, n_features, hidden=64):
        super().__init__()
        self.lstm = nn.LSTM(input_size=n_features, hidden_size=hidden, batch_first=True)
        self.actor = nn.Linear(hidden, 3)
        self.critic = nn.Linear(hidden, 1)
    def forward(self, x):
        out, _ = self.lstm(x)
        h = out[:, -1, :]
        return self.actor(h), self.critic(h)

def a3c_loss(logits, values, actions, rewards, beta=0.01):
    adv = rewards - values.squeeze(-1)
    critic = adv.pow(2).mean()
    logp = torch.log_softmax(logits, dim=-1)
    actor = -(logp.gather(1, actions.unsqueeze(1)).squeeze(1) * adv.detach()).mean()
    entropy = -(torch.softmax(logits, dim=-1) * logp).sum(-1).mean()
    return actor + 0.5*critic - beta*entropy

# --- Train + inference per cluster ---
def process_cluster(meta, epochs=3, lr=1e-3, batch_size=256):
    c_id, tickers, dates = meta["cluster"], meta["tickers"], meta["dates"]
    X = np.load(os.path.join(DATA_DIR, meta["tensor_file"]), mmap_mode="r")
    if X.size == 0:
        return
    B, T, N, F = X.shape
    print(f"Cluster {c_id} | X={X.shape}")

    # rewards (B,N)
    px = []
    for tk in tickers:
        s = df_backtest[df_backtest["ticker"]==tk].set_index("timestamp")["close"]
        s = s.reindex(dates).ffill().bfill().values
        px.append(s)
    px = np.stack(px, axis=1)
    r = np.zeros_like(px, dtype=np.float32)
    r[1:] = np.log(px[1:]/np.maximum(px[:-1],1e-9))

    # model + optimizer
    model = A3CNet(F).to(device)
    opt = optim.Adam(model.parameters(), lr=lr)

    # mini-batch generator
    total = B*N
    def iterator():
        for start in range(0, total, batch_size):
            end = min(total, start+batch_size)
            xb, rb, idx = [], [], []
            for s in range(start,end):
                b, n = divmod(s,N)
                xb.append(X[b,:,n,:]); rb.append(r[b,n]); idx.append((b,n))
            yield np.stack(xb), np.array(rb), idx

    # --- train ---
    for ep in range(epochs):
        loss_ep = 0
        for xb, rb, _ in iterator():
            xb = torch.tensor(xb, dtype=torch.float32).to(device)
            rb = torch.tensor(rb, dtype=torch.float32).to(device)
            act = torch.randint(0,3,(len(xb),),dtype=torch.long).to(device)
            logits, vals = model(xb)
            loss = a3c_loss(logits, vals, act, rb)
            opt.zero_grad(); loss.backward(); opt.step()
            loss_ep += loss.item()
        print(f"  Epoch {ep+1}/{epochs}, Loss={loss_ep:.4f}")
        gc.collect(); torch.cuda.empty_cache()

    # --- save model checkpoint ---
    model_path = os.path.join(MODEL_DIR, f"a3c_cluster_{c_id}.pt")
    torch.save(model.state_dict(), model_path)
    print(f"  ✅ Saved model checkpoint: {model_path}")

    # --- inference & save signals ---
    with open(SIG_FILE,"a",newline="") as f:
        w = csv.writer(f)
        with torch.no_grad():
            for xb, _, idx in iterator():
                xb = torch.tensor(xb,dtype=torch.float32).to(device)
                act = torch.argmax(model(xb)[0], dim=-1).cpu().numpy()-1
                for k,(b,n) in enumerate(idx):
                    w.writerow([dates[b], tickers[n], int(act[k])])
                del xb, act
                gc.collect(); torch.cuda.empty_cache()

    del X, px, r, model, opt
    gc.collect(); torch.cuda.empty_cache()

# --- Run all clusters ---
for meta in tensor_index:
    process_cluster(meta)

print(f"✅ Done Block 8: signals saved to {SIG_FILE}, models in {MODEL_DIR}")


Cluster 0 | X=(490, 64, 21, 22)
  Epoch 1/3, Loss=-0.8344
  Epoch 2/3, Loss=-0.4268
  Epoch 3/3, Loss=-0.4869
  ✅ Saved model checkpoint: ./models/a3c_cluster_0.pt
Cluster 1 | X=(490, 64, 15, 22)
  Epoch 1/3, Loss=-0.6337
  Epoch 2/3, Loss=-0.3487
  Epoch 3/3, Loss=-0.3366
  ✅ Saved model checkpoint: ./models/a3c_cluster_1.pt
Cluster 2 | X=(490, 64, 15, 22)
  Epoch 1/3, Loss=-0.0938
  Epoch 2/3, Loss=-0.2452
  Epoch 3/3, Loss=-0.2638
  ✅ Saved model checkpoint: ./models/a3c_cluster_2.pt
Cluster 3 | X=(490, 64, 17, 22)
  Epoch 1/3, Loss=0.3028
  Epoch 2/3, Loss=-0.2032
  Epoch 3/3, Loss=-0.3266
  ✅ Saved model checkpoint: ./models/a3c_cluster_3.pt
Cluster 4 | X=(490, 64, 19, 22)
  Epoch 1/3, Loss=-0.2497
  Epoch 2/3, Loss=-0.3891
  Epoch 3/3, Loss=-0.3604
  ✅ Saved model checkpoint: ./models/a3c_cluster_4.pt
Cluster 5 | X=(490, 64, 21, 22)
  Epoch 1/3, Loss=0.0066
  Epoch 2/3, Loss=-0.4097
  Epoch 3/3, Loss=-0.4271
  ✅ Saved model checkpoint: ./models/a3c_cluster_5.pt
Cluster 6 | X=(490

In [15]:
# Block 9 — Inference từ checkpoint A3C (load saved models)
import os, gc, json, csv
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

DATA_DIR = "./tensors/"
MODEL_DIR = "./models/"
SIG_DIR   = "./signals/"
os.makedirs(SIG_DIR, exist_ok=True)

SIG_FILE = os.path.join(SIG_DIR, "a3c_signals_infer.csv")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Reset file signals ---
if os.path.exists(SIG_FILE):
    os.remove(SIG_FILE)
with open(SIG_FILE, "w", newline="") as f:
    csv.writer(f).writerow(["date","ticker","signal"])

# --- Load metadata ---
with open(os.path.join(DATA_DIR, "tensor_index.json"), "r") as f:
    tensor_index = json.load(f)

# --- Model định nghĩa lại (giống Block 8) ---
class A3CNet(nn.Module):
    def __init__(self, n_features, hidden=64):
        super().__init__()
        self.lstm = nn.LSTM(input_size=n_features, hidden_size=hidden, batch_first=True)
        self.actor = nn.Linear(hidden, 3)
        self.critic = nn.Linear(hidden, 1)
    def forward(self, x):
        out, _ = self.lstm(x)
        h = out[:, -1, :]
        return self.actor(h), self.critic(h)

# --- Inference function ---
def infer_cluster(meta, batch_size=256):
    c_id, tickers, dates = meta["cluster"], meta["tickers"], meta["dates"]
    X = np.load(os.path.join(DATA_DIR, meta["tensor_file"]), mmap_mode="r")
    if X.size == 0:
        return
    B, T, N, F = X.shape
    print(f"[Inference] Cluster {c_id} | X={X.shape}")

    # load model checkpoint
    model_path = os.path.join(MODEL_DIR, f"a3c_cluster_{c_id}.pt")
    if not os.path.exists(model_path):
        print(f"⚠️ Model checkpoint not found: {model_path}, skip")
        return
    model = A3CNet(F).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # mini-batch inference & save signals
    total = B * N
    with open(SIG_FILE, "a", newline="") as f:
        w = csv.writer(f)
        with torch.no_grad():
            for start in range(0, total, batch_size):
                end = min(total, start+batch_size)
                xb, idx = [], []
                for s in range(start, end):
                    b, n = divmod(s, N)
                    xb.append(X[b, :, n, :])
                    idx.append((b, n))
                xb = torch.tensor(np.stack(xb), dtype=torch.float32).to(device)
                acts = torch.argmax(model(xb)[0], dim=-1).cpu().numpy() - 1
                for k,(b,n) in enumerate(idx):
                    w.writerow([dates[b], tickers[n], int(acts[k])])
                del xb, acts
                gc.collect(); torch.cuda.empty_cache()

    del X, model
    gc.collect(); torch.cuda.empty_cache()

# --- Run inference all clusters ---
for meta in tensor_index:
    infer_cluster(meta)

print(f"✅ Done Block 9: inference signals saved to {SIG_FILE}")

[Inference] Cluster 0 | X=(490, 64, 21, 22)
[Inference] Cluster 1 | X=(490, 64, 15, 22)
[Inference] Cluster 2 | X=(490, 64, 15, 22)
[Inference] Cluster 3 | X=(490, 64, 17, 22)
[Inference] Cluster 4 | X=(490, 64, 19, 22)
[Inference] Cluster 5 | X=(490, 64, 21, 22)
[Inference] Cluster 6 | X=(490, 64, 23, 22)
[Inference] Cluster 7 | X=(490, 64, 25, 22)
[Inference] Cluster 8 | X=(490, 64, 21, 22)
[Inference] Cluster 9 | X=(490, 64, 22, 22)
[Inference] Cluster 10 | X=(490, 64, 23, 22)
[Inference] Cluster 11 | X=(490, 64, 19, 22)
[Inference] Cluster 12 | X=(490, 64, 28, 22)
[Inference] Cluster 13 | X=(490, 64, 29, 22)
[Inference] Cluster 14 | X=(490, 64, 25, 22)
[Inference] Cluster 15 | X=(490, 64, 24, 22)
[Inference] Cluster 16 | X=(490, 64, 25, 22)
[Inference] Cluster 17 | X=(490, 64, 26, 22)
[Inference] Cluster 18 | X=(490, 64, 23, 22)
[Inference] Cluster 19 | X=(490, 64, 25, 22)
[Inference] Cluster 20 | X=(490, 64, 27, 22)
[Inference] Cluster 21 | X=(490, 64, 22, 22)
[Inference] Cluster 

In [ ]:
# Block 10 — Portfolio Construction & Backtest
import pandas as pd
import numpy as np
import os, gc
from FiinQuantX import FiinSession

SIG_DIR = "./signals/"
OUTPUT_DIR = "./backtest/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

INIT_CAPITAL = 10_000  # vốn ban đầu
BENCHMARK = "VNINDEX"

# --- Load signals từ Block 9 ---
a3c_signals = pd.read_csv(os.path.join(SIG_DIR, "a3c_signals_infer.csv"))
a3c_signals["date"] = pd.to_datetime(a3c_signals["date"])

# --- Merge signals với giá close ---
df_merge = pd.merge(
    a3c_signals,
    df_backtest.rename(columns={"timestamp": "date"}),
    on=["date", "ticker"],
    how="left"
).sort_values(["date", "ticker"])

# --- Tính return của từng mã ---
df_merge["return"] = df_merge.groupby("ticker")["close"].pct_change().fillna(0)

# --- Return có trọng số theo tín hiệu ---
df_merge["weighted_ret"] = df_merge["signal"] * df_merge["return"]

# --- Portfolio return = trung bình theo ngày ---
port_ret = df_merge.groupby("date")["weighted_ret"].mean()

# --- Portfolio equity ---
portfolio_value = (1 + port_ret).cumprod() * INIT_CAPITAL

# --- Benchmark: VNINDEX từ API ---
print("🔄 Fetching VNINDEX benchmark...")
username = "DSTC_18@fiinquant.vn"
password = "Fiinquant0606"
client = FiinSession(username=username, password=password).login()

event_history = client.Fetch_Trading_Data(
    realtime=False,
    tickers=BENCHMARK,
    fields=['close'],
    adjusted=True,
    by="1d",
    from_date="2023-01-01"
)

bench_df = event_history.get_data()
bench_df["date"] = pd.to_datetime(bench_df["timestamp"])
bench_df = bench_df.set_index("date")["close"]

bench_ret = bench_df.pct_change().fillna(0)
benchmark_value = (1 + bench_ret).cumprod() * INIT_CAPITAL

print("✅ Done Block 10: Portfolio backtest + benchmark computed.")

# --- Save outputs ---
df_merge.to_csv(os.path.join(OUTPUT_DIR, "signals_full.csv"), index=False)
port_ret.to_frame("port_ret").to_csv(os.path.join(OUTPUT_DIR, "port_ret.csv"))
portfolio_value.to_frame("portfolio_value").to_csv(os.path.join(OUTPUT_DIR, "portfolio_value.csv"))
benchmark_value.to_frame("benchmark_value").to_csv(os.path.join(OUTPUT_DIR, "benchmark_value.csv"))



🔄 Fetching VNINDEX benchmark...
Fetching data, it may take a while. Please wait...
✅ Done Block 10: Portfolio backtest + benchmark computed.


In [17]:
# Block 11 — Walk-Forward Validation
import pandas as pd
import numpy as np
import os, gc
from FiinQuantX import FiinSession

OUTPUT_DIR = "./backtest/"
WFV_DIR = "./walkforward/"
os.makedirs(WFV_DIR, exist_ok=True)

INIT_CAPITAL = 10_000
BENCHMARK = "VNINDEX"

# --- Load signals từ Block 10 ---
df_merge = pd.read_csv(os.path.join(OUTPUT_DIR, "signals_full.csv"))
df_merge["date"] = pd.to_datetime(df_merge["date"])

# --- Load lại df_backtest ---
df_backtest["timestamp"] = pd.to_datetime(df_backtest["timestamp"])

# --- Mốc thời gian WFV ---
train_end = "2024-06-06"
val_end   = "2024-12-31"
test_end  = df_merge["date"].max().strftime("%Y-%m-%d")

splits = {
    "train": (df_merge["date"].min(), train_end),
    "val": (train_end, val_end),
    "test": (val_end, test_end)
}

# --- Hàm backtest theo split ---
def run_backtest(split_name, start, end):
    sub = df_merge[(df_merge["date"] > start) & (df_merge["date"] <= end)].copy()
    if sub.empty:
        print(f"⚠️ No data for {split_name}")
        return

    # return có trọng số
    sub["weighted_ret"] = sub["signal"] * sub.groupby("ticker")["close"].pct_change().fillna(0)

    # portfolio return & equity
    port_ret = sub.groupby("date")["weighted_ret"].mean()
    port_val = (1 + port_ret).cumprod() * INIT_CAPITAL

    # benchmark subset
    bench_sub = bench_ret.loc[start:end]
    bench_val = (1 + bench_sub).cumprod() * INIT_CAPITAL

    # save
    port_ret.to_frame("port_ret").to_csv(os.path.join(WFV_DIR, f"{split_name}_port_ret.csv"))
    port_val.to_frame("portfolio_value").to_csv(os.path.join(WFV_DIR, f"{split_name}_portfolio_value.csv"))
    bench_val.to_frame("benchmark_value").to_csv(os.path.join(WFV_DIR, f"{split_name}_benchmark_value.csv"))
    sub.to_csv(os.path.join(WFV_DIR, f"{split_name}_signals.csv"), index=False)

    print(f"✅ {split_name.capitalize()} split done: {start} → {end}")

# --- Run WFV ---
for split, (start, end) in splits.items():
    run_backtest(split, pd.to_datetime(start), pd.to_datetime(end))




✅ Train split done: 2023-09-14 00:00:00 → 2024-06-06 00:00:00
✅ Val split done: 2024-06-06 00:00:00 → 2024-12-31 00:00:00
✅ Test split done: 2024-12-31 00:00:00 → 2025-08-29 00:00:00


In [19]:
# Block 12 — Visualization & Performance Stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, gc

OUTPUT_DIR = "./backtest/"
WFV_DIR = "./walkforward/"
STATS_FILE = os.path.join(OUTPUT_DIR, "all_stats.csv")

INIT_CAPITAL = 10_000

# --- Helper: max drawdown ---
def max_drawdown(series):
    cummax = series.cummax()
    drawdown = (series - cummax) / cummax
    return drawdown.min()

# --- Helper: performance metrics ---
def compute_stats(port_val, port_ret, bench_val=None):
    stats = {}
    stats["Final Value"] = port_val.iloc[-1]
    stats["ROI (%)"] = (port_val.iloc[-1] / port_val.iloc[0] - 1) * 100
    stats["Sharpe"] = port_ret.mean() / port_ret.std() * np.sqrt(252) if port_ret.std() > 0 else 0
    stats["Sortino"] = port_ret.mean() / port_ret[port_ret < 0].std() * np.sqrt(252) if port_ret[port_ret < 0].std() > 0 else 0
    stats["Max Drawdown (%)"] = max_drawdown(port_val) * 100
    stats["Trades"] = (port_ret != 0).sum()
    stats["Win Rate (%)"] = (port_ret > 0).sum() / max(1, (port_ret != 0).sum()) * 100
    if bench_val is not None:
        stats["Benchmark ROI (%)"] = (bench_val.iloc[-1] / bench_val.iloc[0] - 1) * 100
    return stats

# --- Plotting helper ---
def plot_equity_curve(port_val, bench_val, title, save_path):
    plt.figure(figsize=(10,6))
    plt.plot(port_val.index, port_val.values, label="Portfolio")
    if bench_val is not None:
        plt.plot(bench_val.index, bench_val.values, label="Benchmark (VNINDEX)")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def plot_histogram(port_ret, title, save_path):
    plt.figure(figsize=(8,5))
    plt.hist(port_ret, bins=50, alpha=0.7)
    plt.title(title)
    plt.xlabel("Daily Return")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

# --- Load full backtest (Block 10) ---
# --- Load full backtest (Block 10) ---
port_val = pd.read_csv(os.path.join(OUTPUT_DIR, "portfolio_value.csv"), index_col=0, parse_dates=True).iloc[:,0]
bench_val = pd.read_csv(os.path.join(OUTPUT_DIR, "benchmark_value.csv"), index_col=0, parse_dates=True).iloc[:,0]

# ✅ Tính lại daily return
port_ret = port_val.pct_change().fillna(0)


stats_full = compute_stats(port_val, port_ret, bench_val)
plot_equity_curve(port_val, bench_val, "Full Backtest Equity Curve", os.path.join(OUTPUT_DIR, "equity_full.png"))
plot_histogram(port_ret, "Full Backtest Daily Returns", os.path.join(OUTPUT_DIR, "hist_full.png"))

print("📊 Full Backtest Stats:")
print(pd.Series(stats_full))

# --- Load Walk-Forward splits (Block 11) ---
stats_wfv = {}
for split in ["train", "val", "test"]:
    try:
        pv = pd.read_csv(os.path.join(WFV_DIR, f"{split}_portfolio_value.csv"), index_col=0, parse_dates=True).iloc[:,0]
        pr = pd.read_csv(os.path.join(WFV_DIR, f"{split}_port_ret.csv"), index_col=0, parse_dates=True).iloc[:,0]
        bv = pd.read_csv(os.path.join(WFV_DIR, f"{split}_benchmark_value.csv"), index_col=0, parse_dates=True).iloc[:,0]
        stats_wfv[split] = compute_stats(pv, pr, bv)

        plot_equity_curve(pv, bv, f"{split.capitalize()} Split Equity Curve", os.path.join(WFV_DIR, f"equity_{split}.png"))
        plot_histogram(pr, f"{split.capitalize()} Split Daily Returns", os.path.join(WFV_DIR, f"hist_{split}.png"))
    except Exception as e:
        print(f"⚠️ Skip {split}: {e}")

print("\n📊 Walk-Forward Validation Stats:")
print(pd.DataFrame(stats_wfv))

# --- Special Period Analysis: 26/03/2025 → 15/04/2025 ---
special_start, special_end = "2025-03-26", "2025-04-15"
sub_val = port_val.loc[special_start:special_end]
sub_ret = port_ret.loc[special_start:special_end]
sub_bench = bench_val.loc[special_start:special_end]

stats_special = {}
if not sub_val.empty:
    stats_special = compute_stats(sub_val, sub_ret, sub_bench)
    plot_equity_curve(sub_val, sub_bench, "Special Period Equity Curve (Tax Shock)", os.path.join(OUTPUT_DIR, "equity_special.png"))
    plot_histogram(sub_ret, "Special Period Daily Returns", os.path.join(OUTPUT_DIR, "hist_special.png"))

    print("\n📊 Special Period (26/03/2025 → 15/04/2025) Stats:")
    print(pd.Series(stats_special))
else:
    print("⚠️ No data in special period (26/03/2025 → 15/04/2025)")

# --- Save all stats to CSV ---
all_stats = {"full": stats_full}
all_stats.update({f"wfv_{k}": v for k,v in stats_wfv.items()})
if stats_special:
    all_stats["special"] = stats_special

df_stats = pd.DataFrame(all_stats).T
df_stats.to_csv(STATS_FILE)
print(f"\n✅ All stats saved to {STATS_FILE}")


📊 Full Backtest Stats:
Final Value          10029.894326
ROI (%)                  0.298943
Sharpe                   0.542019
Sortino                  0.905769
Max Drawdown (%)        -0.204617
Trades                 489.000000
Win Rate (%)            49.693252
Benchmark ROI (%)       61.146662
dtype: float64

📊 Walk-Forward Validation Stats:
                          train           val         test
Final Value        10021.122617  10013.570994  9995.183885
ROI (%)                0.211226      0.135710    -0.048161
Sharpe                 0.914405      0.907077    -0.287799
Sortino                1.410890      1.741976    -0.496889
Max Drawdown (%)      -0.184534     -0.108936    -0.197115
Trades               179.000000    145.000000   162.000000
Win Rate (%)          53.072626     48.275862    46.913580
Benchmark ROI (%)      4.882294     -1.307302    32.794171

📊 Special Period (26/03/2025 → 15/04/2025) Stats:
Final Value          10047.490851
ROI (%)                  0.040172
Sharpe